Yaprak Sınıflandırması
Gerekli Kütüphanelerin Eklenmesi

In [94]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


Test ve Eğitim Verilerinin Okunması


In [95]:
train=pd.read_csv("C:/Users/omerk/Documents/GitHub/yapayzeka/hafta8/yaprakveriseti/train.csv")
test=pd.read_csv("C:/Users/omerk/Documents/GitHub/yapayzeka/hafta8/yaprakveriseti/test.csv")

Sınıfların Belirlenmesi ve Etiketlenmesi


In [96]:
label_encoder=LabelEncoder().fit(train.species)
labels=label_encoder.transform(train.species)
classes =list(label_encoder.classes_)

Verilerin Hazırlanması , Özellik ve Sınıf Sayısının Belirlenmesi 

In [97]:
train=train.drop(["id","species"],axis=1)
test=test.drop(["id"],axis=1)
nb_features=192
nb_classes=len(classes)

Eğitim Verilerinin Standartlaştırılması

In [98]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(train.values)
train=scaler.transform(train.values)

Eğitim Verisinin Eğitim ve Doğrulama İçin Ayarlanması

In [99]:
from sklearn.model_selection import train_test_split
x_train , x_valid , y_train , y_valid = train_test_split(train, labels ,test_size=0.1)

Etiketlerin kategorileştirilmesi


In [101]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)

x_train=np.array(x_train).reshape(891,192,1)
x_valid=np.array(x_valid).reshape(99,192,1)

Modelin oluşturulması 


In [102]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Conv1D,Dropout,MaxPooling1D,Flatten

model=Sequential()
model.add(Conv1D(512,1,input_shape=(nb_features,1)))
model.add(Activation("relu"))
model.add(MaxPooling1D(2))
model.add(Conv1D(256,1))
model.add(Activation("relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2048,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(nb_classes,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 192, 512)          1024      
                                                                 
 activation (Activation)     (None, 192, 512)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 96, 512)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 96, 256)           131328    
                                                                 
 activation_1 (Activation)   (None, 96, 256)           0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 48, 256)          0         
 1D)                                                    

Ağın Derlenmesi

In [103]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Modelin Eğitilmesi


In [104]:
model.fit(x_train,y_train,epochs=15,validation_data=(x_valid,y_valid))

Epoch 1/75


ValueError: in user code:

    File "C:\Users\omerk\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\omerk\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\omerk\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\omerk\anaconda3\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\omerk\anaconda3\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\omerk\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "